In [1]:
%pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 108.6 MB/s eta 0:00:0000:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="1GlZktR4NmlZwEPbwo8V")
project = rf.workspace("gurukul").project("attendance-system-using-face")
version = project.version(2)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Attendance-System-Using-Face--2 in coco:: 100%|██████████| 734/734 [00:00<00:00, 3227.17it/s]


In [3]:
print(dataset.location)

/content/Attendance-System-Using-Face--2


### Checking Contents

In [4]:
import os

root_dir = "/content/Attendance-System-Using-Face--2"

print("Root contents:", os.listdir(root_dir))
print("Train contents:", os.listdir(os.path.join(root_dir, "train")))


Root contents: ['test', 'train', 'valid', 'README.roboflow.txt', 'README.dataset.txt']
Train contents: ['1-64-_jpg.rf.da8a456ab866c610c961d5cdf004701b.jpg', '1683268848381_1683268825008_jpg.rf.8097b82a75b92f5133e5cae092ed01c9.jpg', 'IMG20230505113443_jpg.rf.71ee97d964b94b7c6b919ca4b1cd4144.jpg', 'IMG20230505114756_jpg.rf.89f2450d2f6e805a33a7ee7cb137fb76.jpg', 'IMG_4751_JPG.rf.5fca8a32604d71cfaa01b752b2c384b4.jpg', 'IMG20230505114435_jpg.rf.35b564bc026cd1482bc74f3d6cec6676.jpg', 'IMG20230505114148_jpg.rf.df3039a131e4d3ca0db387cb21473bfd.jpg', '1683268849649_1683268825250_jpg.rf.ce6f57b8c7c3e02bd067fd03f893898f.jpg', 'IMG20230505114124_jpg.rf.028fde97a16152dc1f00c8cfd154e39d.jpg', 'IMG20230505114109_jpg.rf.0a5948697db93c06da37f32deee2fb83.jpg', '1-70-_jpg.rf.957550a9f071fe9ee02ef6fb9900ab2a.jpg', 'IMG20230505113538_jpg.rf.98ef4f91c06fae749a1ebdb6e723ae8a.jpg', 'IMG20230505114014_jpg.rf.552ac30ad4f9b47a263b00e520e74e3d.jpg', 'IMG20230505113321_jpg.rf.fd81317cf1a7d15c64f5d5959fd4a913.jpg',

### Crop faces from COCO for all splits

In [5]:
import json
import os
from pathlib import Path
from PIL import Image

root_dir = Path("/content/Attendance-System-Using-Face--2")

def crop_split(split_name: str):
    split_dir = root_dir / split_name
    ann_path = split_dir / "_annotations.coco.json"

    if not ann_path.exists():
        print(f"[{split_name}] No COCO annotation file found at {ann_path}, skipping.")
        return

    out_dir = root_dir / f"cropped_faces_{split_name}"
    out_dir.mkdir(exist_ok=True)

    with open(ann_path, "r") as f:
        coco = json.load(f)

    images = {img["id"]: img for img in coco["images"]}
    annotations = coco["annotations"]

    print(f"[{split_name}] Found {len(images)} images, {len(annotations)} face instances")

    counter = 0
    for ann in annotations:
        img_id = ann["image_id"]
        img_info = images[img_id]
        file_name = img_info["file_name"]

        img_path = split_dir / file_name
        if not img_path.exists():
            continue

        # COCO bbox = [x, y, width, height]
        x, y, w, h = ann["bbox"]
        x, y, w, h = int(x), int(y), int(w), int(h)

        with Image.open(img_path) as im:
            # clip to image bounds
            x2 = min(x + w, im.width)
            y2 = min(y + h, im.height)
            x = max(x, 0)
            y = max(y, 0)

            if x2 <= x or y2 <= y:
                continue

            face_crop = im.crop((x, y, x2, y2))

            out_file = out_dir / f"face_{counter:06d}.jpg"
            face_crop.save(out_file)
            counter += 1

    print(f"[{split_name}] Saved {counter} cropped faces to: {out_dir}")

# Run for train / valid / test
for split in ["train", "valid", "test"]:
    crop_split(split)


[train] Found 633 images, 1857 face instances
[train] Saved 1857 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_train
[valid] Found 62 images, 183 face instances
[train] Saved 1857 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_train
[valid] Found 62 images, 183 face instances
[valid] Saved 183 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_valid
[test] Found 31 images, 97 face instances
[valid] Saved 183 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_valid
[test] Found 31 images, 97 face instances
[test] Saved 97 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_test
[test] Saved 97 cropped faces to: /content/Attendance-System-Using-Face--2/cropped_faces_test


### Confirm cropped faces exist

In [6]:
import os

print("Cropped train faces:", len(os.listdir("/content/Attendance-System-Using-Face--2/cropped_faces_train")))


Cropped train faces: 1857


### Data Preparation & Augmentation

In [7]:
from pathlib import Path

ROOT = Path("/content/Attendance-System-Using-Face--2")

cropped_train = ROOT / "cropped_faces_train"
cropped_valid = ROOT / "cropped_faces_valid"
cropped_test  = ROOT / "cropped_faces_test"

print("Train dir exists:", cropped_train.exists())
print("Valid dir exists:", cropped_valid.exists())
print("Test dir exists:",  cropped_test.exists())


Train dir exists: True
Valid dir exists: True
Test dir exists: True


In [8]:
%pip install opencv-python pillow dlib

### Defining Blurring & Noising Augmentation Functions

In [9]:
import cv2
import numpy as np
from pathlib import Path
from PIL import Image

def add_gaussian_blur(img, ksize=5):
    # ksize must be odd: 3,5,7,...
    return cv2.GaussianBlur(img, (ksize, ksize), 0)

def add_gaussian_noise(img, mean=0, var=20):
    # img: uint8 BGR
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, img.shape).reshape(img.shape).astype(np.float32)
    noisy = img.astype(np.float32) + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def add_salt_pepper_noise(img, amount=0.01):
    # amount = fraction of pixels to corrupt
    noisy = img.copy()
    num_pixels = img.shape[0] * img.shape[1]
    num_salt = int(amount * num_pixels / 2)
    num_pepper = int(amount * num_pixels / 2)

    # Salt (white)
    coords = (
        np.random.randint(0, img.shape[0], num_salt),
        np.random.randint(0, img.shape[1], num_salt),
    )
    noisy[coords] = 255

    # Pepper (black)
    coords = (
        np.random.randint(0, img.shape[0], num_pepper),
        np.random.randint(0, img.shape[1], num_pepper),
    )
    noisy[coords] = 0

    return noisy


### Augment all cropped faces

In [10]:
import os
from tqdm import tqdm

def augment_split(split_name: str):
    in_dir = ROOT / f"cropped_faces_{split_name}"
    if not in_dir.exists():
        print(f"[{split_name}] Input dir not found, skipping:", in_dir)
        return

    out_dir = ROOT / f"aug_faces_{split_name}"
    out_dir.mkdir(exist_ok=True)

    images = [f for f in os.listdir(in_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    print(f"[{split_name}] Found {len(images)} images in {in_dir}")

    counter = 0
    for fname in tqdm(images, desc=f"Augmenting {split_name}"):
        in_path = in_dir / fname
        img = cv2.imread(str(in_path))
        if img is None:
            continue

        # Save original copy
        base_name = Path(fname).stem
        ext = Path(fname).suffix

        cv2.imwrite(str(out_dir / f"{base_name}_orig{ext}"), img)

        # Blurs with different kernel sizes
        for k in [3, 5, 7]:
            blurred = add_gaussian_blur(img, ksize=k)
            cv2.imwrite(str(out_dir / f"{base_name}_blur{k}{ext}"), blurred)

        # Gaussian noise with different variances
        for v in [10, 20, 30]:
            noisy = add_gaussian_noise(img, var=v)
            cv2.imwrite(str(out_dir / f"{base_name}_gn{v}{ext}"), noisy)

        # Salt & pepper noise
        for amt in [0.01, 0.02]:
            sp = add_salt_pepper_noise(img, amount=amt)
            cv2.imwrite(str(out_dir / f"{base_name}_sp{int(amt*100)}{ext}"), sp)

        counter += 1

    print(f"[{split_name}] Augmentation done. Check:", out_dir)

for split in ["train", "valid", "test"]:
    augment_split(split)


[train] Found 1857 images in /content/Attendance-System-Using-Face--2/cropped_faces_train


[train] Found 1857 images in /content/Attendance-System-Using-Face--2/cropped_faces_train


Augmenting train: 100%|██████████| 1857/1857 [00:10<00:00, 171.61it/s]


[train] Found 1857 images in /content/Attendance-System-Using-Face--2/cropped_faces_train


Augmenting train: 100%|██████████| 1857/1857 [00:10<00:00, 171.61it/s]


...

[train] Augmentation done. Check: /content/Attendance-System-Using-Face--2/aug_faces_train
[valid] Found 183 images in /content/Attendance-System-Using-Face--2/cropped_faces_valid


Augmenting valid: 100%|██████████| 183/183 [00:01<00:00, 152.16it/s]



[valid] Augmentation done. Check: /content/Attendance-System-Using-Face--2/aug_faces_valid
[test] Found 97 images in /content/Attendance-System-Using-Face--2/cropped_faces_test


Augmenting test: 100%|██████████| 97/97 [00:00<00:00, 137.56it/s]

[test] Augmentation done. Check: /content/Attendance-System-Using-Face--2/aug_faces_test


In [12]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
!mkdir -p /content/models
!mv shape_predictor_68_face_landmarks.dat /content/models/

--2025-12-03 09:57:14--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-12-03 09:57:14--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  25.2MB/s    in 2.4s    

2025-12-03 09:57:17 (25.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



### DLib Library Setup

In [13]:
import dlib

shape_model_path = "/content/models/shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(shape_model_path)

print("dlib landmark model loaded successfully ")


dlib landmark model loaded successfully 


### Setup paths

In [14]:
from pathlib import Path

ROOT = Path("/content/Attendance-System-Using-Face--2")

cropped_train = ROOT / "cropped_faces_train"
cropped_valid = ROOT / "cropped_faces_valid"
cropped_test  = ROOT / "cropped_faces_test"

print("Train dir:", cropped_train, "exists:", cropped_train.exists())
print("Valid dir:", cropped_valid, "exists:", cropped_valid.exists())
print("Test dir:",  cropped_test,  "exists:", cropped_test.exists())


Train dir: /content/Attendance-System-Using-Face--2/cropped_faces_train exists: True
Valid dir: /content/Attendance-System-Using-Face--2/cropped_faces_valid exists: True
Test dir: /content/Attendance-System-Using-Face--2/cropped_faces_test exists: True


### Define face alignment function using dlib landmarks

In [16]:
import cv2
import dlib
import numpy as np
from pathlib import Path
import os

ROOT = Path("/content/Attendance-System-Using-Face--2")

# Path to your Kazemi-style landmark model
shape_model_path = "/content/models/shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(shape_model_path)

print("dlib ERT landmark model loaded ")


dlib ERT landmark model loaded 


In [17]:
def align_face_dlib(img_bgr, desired_size=160, desired_left_eye=(0.35, 0.35)):
    """
    Aligns a face using dlib's 68-point landmarks (Kazemi & Sullivan ERT)
    and an affine transform based on eye positions.

    img_bgr: OpenCV image (BGR)
    returns: aligned face (BGR) or None if no face found
    """
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    dets = detector(img_rgb, 1)

    if len(dets) == 0:
        return None

    # Pick the largest detected face (in case there are multiple)
    rect = max(dets, key=lambda r: r.width() * r.height())
    shape = shape_predictor(img_rgb, rect)

    # 68 landmarks → numpy
    landmarks = np.array([[p.x, p.y] for p in shape.parts()], dtype=np.float32)

    # Eye landmark indices for the 68-point model
    left_eye_pts  = landmarks[36:42]
    right_eye_pts = landmarks[42:48]

    left_eye_center  = left_eye_pts.mean(axis=0)
    right_eye_center = right_eye_pts.mean(axis=0)

    # Compute angle between eye centers
    dY = right_eye_center[1] - left_eye_center[1]
    dX = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Compute scale based on desired eye distance
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desired_right_eye_x = 1.0 - desired_left_eye[0]
    desired_dist = (desired_right_eye_x - desired_left_eye[0]) * desired_size
    scale = desired_dist / dist

    # Eye center midpoint
    eyes_center = ((left_eye_center[0] + right_eye_center[0]) / 2,
                   (left_eye_center[1] + right_eye_center[1]) / 2)

    # Get affine rotation+scale matrix
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

    # Shift so that eyes land in the desired position
    tX = desired_size * 0.5
    tY = desired_size * desired_left_eye[1]
    M[0, 2] += (tX - eyes_center[0])
    M[1, 2] += (tY - eyes_center[1])

    # Apply affine transform
    aligned = cv2.warpAffine(img_bgr, M, (desired_size, desired_size),
                             flags=cv2.INTER_CUBIC)

    return aligned


In [18]:
from tqdm import tqdm

def align_split(split_name: str):
    in_dir = ROOT / f"cropped_faces_{split_name}"
    if not in_dir.exists():
        print(f"[{split_name}] Input dir not found, skipping:", in_dir)
        return

    out_dir = ROOT / f"aligned_faces_{split_name}"
    out_dir.mkdir(exist_ok=True)

    files = [f for f in os.listdir(in_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    print(f"[{split_name}] Found {len(files)} input face crops")

    aligned_count = 0
    skipped = 0
    for fname in tqdm(files, desc=f"Aligning {split_name}"):
        in_path = in_dir / fname
        img = cv2.imread(str(in_path))
        if img is None:
            skipped += 1
            continue

        aligned = align_face_dlib(img)
        if aligned is None:
            skipped += 1
            continue

        out_path = out_dir / fname
        cv2.imwrite(str(out_path), aligned)
        aligned_count += 1

    print(f"[{split_name}] Aligned: {aligned_count}, Skipped: {skipped}. Saved to: {out_dir}")

for split in ["train", "valid", "test"]:
    align_split(split)


[train] Found 1857 input face crops


Aligning train: 100%|██████████| 1857/1857 [00:20<00:00, 89.27it/s] 


[train] Aligned: 1035, Skipped: 822. Saved to: /content/Attendance-System-Using-Face--2/aligned_faces_train
[valid] Found 183 input face crops


Aligning valid: 100%|██████████| 183/183 [00:01<00:00, 109.52it/s]


[valid] Aligned: 92, Skipped: 91. Saved to: /content/Attendance-System-Using-Face--2/aligned_faces_valid
[test] Found 97 input face crops


Aligning test: 100%|██████████| 97/97 [00:00<00:00, 120.52it/s]

[test] Aligned: 47, Skipped: 50. Saved to: /content/Attendance-System-Using-Face--2/aligned_faces_test


In [19]:
for split in ["train", "valid", "test"]:
    out_dir = ROOT / f"aligned_faces_{split}"
    if out_dir.exists():
        print(split, "=>", len(os.listdir(out_dir)), "aligned images")
    else:
        print(split, "=> no aligned dir")


train => 1035 aligned images
valid => 92 aligned images
test => 47 aligned images


In [20]:
from pathlib import Path

ROOT = Path("/content/Attendance-System-Using-Face--2")

aligned_train = ROOT / "aligned_faces_train"
aligned_valid = ROOT / "aligned_faces_valid"
aligned_test  = ROOT / "aligned_faces_test"

print("Train aligned:", aligned_train, "exists:", aligned_train.exists())
print("Valid aligned:", aligned_valid, "exists:", aligned_valid.exists())
print("Test aligned:",  aligned_test,  "exists:", aligned_test.exists())


Train aligned: /content/Attendance-System-Using-Face--2/aligned_faces_train exists: True
Valid aligned: /content/Attendance-System-Using-Face--2/aligned_faces_valid exists: True
Test aligned: /content/Attendance-System-Using-Face--2/aligned_faces_test exists: True


### Augment Aligned Faces

In [21]:
import os
from pathlib import Path
from tqdm import tqdm
import cv2

ROOT = Path("/content/Attendance-System-Using-Face--2")

def augment_aligned_split(split_name: str):
    in_dir = ROOT / f"aligned_faces_{split_name}"
    if not in_dir.exists():
        print(f"[{split_name}] Input dir not found, skipping:", in_dir)
        return

    out_dir = ROOT / f"aug_aligned_faces_{split_name}"
    out_dir.mkdir(exist_ok=True)

    images = [f for f in os.listdir(in_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    print(f"[{split_name}] Found {len(images)} aligned images in {in_dir}")

    for fname in tqdm(images, desc=f"Augmenting aligned {split_name}"):
        in_path = in_dir / fname
        img = cv2.imread(str(in_path))
        if img is None:
            continue

        base_name = Path(fname).stem
        ext = Path(fname).suffix

        # 1) Save original aligned copy
        cv2.imwrite(str(out_dir / f"{base_name}_orig{ext}"), img)

        # 2) Blurs with different kernel sizes
        for k in [3, 5, 7]:
            blurred = add_gaussian_blur(img, ksize=k)
            cv2.imwrite(str(out_dir / f"{base_name}_blur{k}{ext}"), blurred)

        # 3) Gaussian noise with different variances
        for v in [10, 20, 30]:
            noisy = add_gaussian_noise(img, var=v)
            cv2.imwrite(str(out_dir / f"{base_name}_gn{v}{ext}"), noisy)

        # 4) Salt & pepper noise
        for amt in [0.01, 0.02]:
            sp = add_salt_pepper_noise(img, amount=amt)
            cv2.imwrite(str(out_dir / f"{base_name}_sp{int(amt*100)}{ext}"), sp)

    print(f"[{split_name}] Augmentation complete. Check folder: {out_dir}")

for split in ["train", "valid", "test"]:
    augment_aligned_split(split)


[train] Found 1035 aligned images in /content/Attendance-System-Using-Face--2/aligned_faces_train


Augmenting aligned train: 100%|██████████| 1035/1035 [00:13<00:00, 77.07it/s]


[train] Augmentation complete. Check folder: /content/Attendance-System-Using-Face--2/aug_aligned_faces_train
[valid] Found 92 aligned images in /content/Attendance-System-Using-Face--2/aligned_faces_valid


Augmenting aligned valid: 100%|██████████| 92/92 [00:01<00:00, 82.80it/s]


[valid] Augmentation complete. Check folder: /content/Attendance-System-Using-Face--2/aug_aligned_faces_valid
[test] Found 47 aligned images in /content/Attendance-System-Using-Face--2/aligned_faces_test


Augmenting aligned test: 100%|██████████| 47/47 [00:00<00:00, 80.32it/s]

[test] Augmentation complete. Check folder: /content/Attendance-System-Using-Face--2/aug_aligned_faces_test


In [22]:
for split in ["train", "valid", "test"]:
    out_dir = ROOT / f"aug_aligned_faces_{split}"
    if out_dir.exists():
        print(split, "=>", len(os.listdir(out_dir)), "augmented aligned images")
    else:
        print(split, "=> no aug_aligned dir")


train => 9315 augmented aligned images
valid => 828 augmented aligned images
test => 423 augmented aligned images


In [26]:
%pip install facenet-pytorch torch torchvision

In [28]:
!pip install facenet-pytorch

### Load FaceNet Model

In [31]:
import torch
from facenet_pytorch.models.inception_resnet_v1 import InceptionResnetV1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

model = InceptionResnetV1(pretrained='vggface2', classify=False).eval().to(device)
print("FaceNet-like model loaded ")


Using device: cpu
FaceNet-like model loaded 


In [1]:
!pip install PILLOW

In [2]:
# Ensure a compatible PyTorch + torchvision is installed before importing `transforms`
# This attempts to install CPU wheels from the official PyTorch index if imports fail.
import sys, subprocess, importlib
def pip_install(args):
    subprocess.check_call([sys.executable, "-m", "pip"] + args)
try:
    import torch, torchvision
    print("Found torch", torch.__version__, "torchvision", torchvision.__version__)
except Exception as e:
    print("torch/torchvision import failed (or missing). Installing compatible builds (CPU wheels). This may take a few minutes...")
    pip_install(["install", "--upgrade", "pip"])
    # Use the official PyTorch CPU wheels index which is widely compatible.
    try:
        pip_install(["install", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cpu"])
    except Exception:
        # Fall back to a simpler install if the index approach fails
        pip_install(["install", "torch", "torchvision"])
    importlib.invalidate_caches()
    torch = importlib.import_module("torch")
    torchvision = importlib.import_module("torchvision")
    print("Installed torch", torch.__version__, "torchvision", torchvision.__version__)

Found torch 2.2.2+cu121 torchvision 0.17.2+cu121


### Building Dataset for FaceNet 

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path

ROOT = Path("/content/Attendance-System-Using-Face--2")
train_aug_dir = ROOT / "aug_aligned_faces_train"

class FaceImageFolder(Dataset):
    def __init__(self, folder: Path):
        self.folder = folder
        self.paths = sorted(
            [
                folder / f
                for f in os.listdir(folder)
                if f.lower().endswith((".jpg", ".jpeg", ".png"))
            ]
        )
        self.transform = transforms.Compose([
            transforms.Resize((160, 160)),
            transforms.ToTensor(),                      # [0,1]
            transforms.Normalize([0.5, 0.5, 0.5],       # [-1,1]
                                 [0.5, 0.5, 0.5]),
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, str(path)

train_dataset = FaceImageFolder(train_aug_dir)
print("Train images:", len(train_dataset))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)


Train images: 9315


In [4]:
# Build validation dataset and dataloader (mirrors train)
valid_aug_dir = ROOT / "aug_aligned_faces_valid"
if not valid_aug_dir.exists():
    print("Validation augmented folder not found:", valid_aug_dir)
valid_dataset = FaceImageFolder(valid_aug_dir)
print("Valid images:", len(valid_dataset))
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=2)

Valid images: 828


In [5]:
# Build test dataset and dataloader
test_aug_dir = ROOT / "aug_aligned_faces_test"
if not test_aug_dir.exists():
    print("Test augmented folder not found:", test_aug_dir)
test_dataset = FaceImageFolder(test_aug_dir)
print("Test images:", len(test_dataset))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Test images: 423


In [7]:
import numpy as np
from torch.utils.data import DataLoader
from pathlib import Path
import os
from PIL import Image
import torch

ROOT = Path("/content/Attendance-System-Using-Face--2")

def preprocess_img(img: Image.Image):
    img = img.resize((160, 160))
    img_np = torch.from_numpy((np.array(img)).astype("float32"))  # [H, W, C]
    img_np = img_np.permute(2, 0, 1) / 255.0                      # [C, H, W]
    img_np = (img_np - 0.5) / 0.5                                 # [-1, 1]
    return img_np

class FaceImageFolder(torch.utils.data.Dataset):
    def __init__(self, folder: Path):
        self.folder = folder
        self.paths = sorted(
            [
                folder / f
                for f in os.listdir(folder)
                if f.lower().endswith((".jpg", ".jpeg", ".png"))
            ]
        )

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        img_t = preprocess_img(img)
        return img_t, str(path)


def compute_embeddings_for_folder(folder: Path, out_name: str, model, device="cuda"):
    dataset = FaceImageFolder(folder)
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

    all_embs = []
    all_paths = []
    model.eval()

    with torch.no_grad():
        for imgs, batch_paths in loader:
            imgs = imgs.to(device)
            embs = model(imgs)              # (B, 512)
            all_embs.append(embs.cpu().numpy())
            all_paths.extend(batch_paths)

    all_embs = np.concatenate(all_embs, axis=0)
    all_paths = np.array(all_paths)

    out_path = ROOT / out_name
    np.savez(out_path, paths=all_paths, embeddings=all_embs)
    print(f"Saved {all_embs.shape[0]} embeddings to: {out_path}")


In [8]:
from facenet_pytorch.models.inception_resnet_v1 import InceptionResnetV1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = InceptionResnetV1(pretrained='vggface2', classify=False).eval().to(device)


In [9]:
compute_embeddings_for_folder(ROOT / "aug_aligned_faces_train", "train_facenet_embeddings.npz", model, device)
compute_embeddings_for_folder(ROOT / "aug_aligned_faces_valid", "valid_facenet_embeddings.npz", model, device)
compute_embeddings_for_folder(ROOT / "aug_aligned_faces_test",  "test_facenet_embeddings.npz",  model, device)

KeyboardInterrupt: 